In [2]:
# !pip install transformers yake

In [47]:
from transformers import pipeline
import yake

In [10]:
import time
import csv
from functools import wraps
from typing import Callable

def log_runtime_to_csv(n: int, filename: str) -> Callable:
    """
    Decorator to run a function `n` times, log each runtime to a CSV file, and calculate the average runtime.
    
    Args:
        n (int): Number of times to run the decorated function.
        filename (str): Path to the CSV file where runtimes will be logged.
        
    Returns:
        Callable: The decorated function.
    """
    
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            runtimes = []
            for _ in range(n):
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                runtime = end_time - start_time
                runtimes.append(runtime)
                
                # Log runtime to CSV
                with open(filename, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([runtime])
            
            # Calculate average runtime
            average_runtime = sum(runtimes) / len(runtimes)
            print(f"Average runtime over {n} runs: {average_runtime:.4f} seconds")
            
            return result
        return wrapper
    return decorator


In [11]:
def analyze_sentiment(review):
    sentiment_analyzer = pipeline("sentiment-analysis")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5):
    
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=num_keywords, dedupLim=0.9, top=5, features=None)
    keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    return keywords

def extract_issues(text):
    
    issues = []
    if "bug" in text.lower():
        issues.append("bug")
    if "problem" in text.lower():
        issues.append("problem")
    if "issue" in text.lower():
        issues.append("issue")
    return issues

@log_runtime_to_csv(n=10, filename='original_approach_runtimes.csv')
def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        sentiment_result = analyze_sentiment(review)
        sentiment = sentiment_result['label']
        score = sentiment_result['score']

        if sentiment == 'POSITIVE':
            
            positive_aspects.update(extract_keywords(review))
        elif sentiment == 'NEGATIVE':
            
            negative_aspects.update(extract_keywords(review))
            issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)


reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]


positive_aspects, negative_aspects, issues = extract_insights(reviews)


print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision a

Average runtime over 10 runs: 23.2101 seconds
Positive Aspects: ['expectations', 'Excellent product', 'Affordable', 'durable and built', 'exceeded my expectations', 'interface is intuitive', 'manual is comprehensive and easy', 'packaging', 'Highly recommended', 'offers', 'product', 'team was helpful and resolved', 'Fantastic customer', 'service', 'Fantastic', 'versatile', 'amazing', 'customer support team', 'eco-friendly', 'customer service', 'job', 'packaging was eco-friendly', 'applications', 'product exceeded my expectations', 'price', 'product arrived earlier than expected', 'product is a game-changer', 'easy to follow', 'intuitive', 'quality is excellent', 'sound quality', 'build quality', 'Average', 'user interface is intuitive', 'customer support team was helpful', 'features it offers', 'life', 'game-changer', 'Excellent', 'worth', 'love', 'product is durable and built', 'Impressed with the build quality', 'user manual is comprehensive', 'product arrived', 'reasonable considerin

In [1]:
from transformers import pipeline
import yake
import spacy
from collections import defaultdict

# Load the spaCy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

def analyze_sentiment(review):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5):
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=3, dedupLim=0.9, top=num_keywords, features=None)
    keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    return keywords

def extract_issues(text):
    issues = []
    doc = nlp(text.lower())
    for token in doc:
        if token.lemma_ in ["bug", "problem", "issue", "crash"]:
            issues.append(token.text)
    return issues

def context_aware_sentiment_classification(text):
    doc = nlp(text)
    sentiments = defaultdict(list)
    
    for chunk in doc.noun_chunks:
        sentiment_result = analyze_sentiment(chunk.text)
        sentiment = sentiment_result['label']
        sentiments[sentiment].append(chunk.text)
    
    return sentiments

def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        context_sentiments = context_aware_sentiment_classification(review)
        
        positive_aspects.update(context_sentiments['POSITIVE'])
        negative_aspects.update(context_sentiments['NEGATIVE'])
        issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)


# Example reviews
reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]

# Extract insights
positive_aspects, negative_aspects, issues = extract_insights(reviews)

# Output results
print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

c:\Users\Asif Sayyed\Documents\GitHub\MachineUnderstandsTextData\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asif Sayyed\Documents\GitHub\MachineUnderstandsTextData\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asif Sayyed\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you eit

Positive Aspects: ['Excellent product', 'It', 'This product', 'the job', 'the performance', 'the customer service', 'me', 'the user experience', 'I', 'that', 'functionality', 'The interface', 'the return process', 'Great value', 'a plus', 'it', 'my issue', 'eco', 'life', 'The price', 'The quality', 'the features', 'immediate attention', 'Pleasant surprise', 'The sound quality', 'The customer support team', 'the build quality', 'The user interface', 'They', 'the battery', 'this product', 'the money', 'the device', 'The packaging', 'The product', 'the shipping time', 'various applications', 'a game-changer', 'Fantastic customer service', 'high-quality', 'The design', 'which', 'Great features', 'some advanced features', 'my expectations', 'the price', 'a modern redesign', 'The instructions']
Negative Aspects: ['The user manual', 'Too many unnecessary features', 'Laggy', 'Nothing', 'Terrible experience', 'Average product', 'the battery life', 'Poor packaging', 'a damaged product', 'a hassl

In [2]:
from transformers import pipeline
import yake
import spacy
from collections import defaultdict

# Load the spaCy model for dependency parsing and stopwords
nlp = spacy.load("en_core_web_sm")
stopwords = spacy.lang.en.stop_words.STOP_WORDS

def analyze_sentiment(review):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5, min_length=3):
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=3, dedupLim=0.9, top=num_keywords, features=None)
    raw_keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    
    # Filter out keywords that are stopwords or below the minimum length
    filtered_keywords = [
        kw for kw in raw_keywords 
        if kw.lower() not in stopwords and len(kw) >= min_length
    ]
    
    return filtered_keywords

def extract_issues(text):
    issues = []
    doc = nlp(text.lower())
    for token in doc:
        if token.lemma_ in ["bug", "problem", "issue", "crash"]:
            issues.append(token.text)
    return issues

def context_aware_sentiment_classification(text):
    doc = nlp(text)
    sentiments = defaultdict(list)
    
    for chunk in doc.noun_chunks:
        # Filter out noun chunks that are just stopwords
        if chunk.root.text.lower() in stopwords:
            continue
        
        sentiment_result = analyze_sentiment(chunk.text)
        sentiment = sentiment_result['label']
        sentiments[sentiment].append(chunk.text)
    
    return sentiments

def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        context_sentiments = context_aware_sentiment_classification(review)
        
        positive_aspects.update(context_sentiments['POSITIVE'])
        negative_aspects.update(context_sentiments['NEGATIVE'])
        issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)

# Example reviews
reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]

# Extract insights
positive_aspects, negative_aspects, issues = extract_insights(reviews)

# Output results
print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

Positive Aspects: ['Excellent product', 'This product', 'the job', 'the performance', 'the customer service', 'the user experience', 'functionality', 'The interface', 'the return process', 'Great value', 'a plus', 'my issue', 'eco', 'life', 'The price', 'The quality', 'the features', 'immediate attention', 'Pleasant surprise', 'The sound quality', 'The customer support team', 'the build quality', 'The user interface', 'the battery', 'this product', 'the money', 'the device', 'The packaging', 'The product', 'the shipping time', 'various applications', 'a game-changer', 'Fantastic customer service', 'high-quality', 'The design', 'Great features', 'some advanced features', 'my expectations', 'the price', 'a modern redesign', 'The instructions']
Negative Aspects: ['The user manual', 'Too many unnecessary features', 'Laggy', 'Terrible experience', 'Average product', 'the battery life', 'Poor packaging', 'a damaged product', 'a hassle', 'Numerous issues', 'a software bug']
Issues: ['bug', 'c

In [5]:
from transformers import pipeline
import yake

def analyze_sentiment(review: str):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5):
    # Using the initial keyword extraction approach
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=5, dedupLim=0.9, top=num_keywords, features=None)
    keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    return keywords

def extract_issues(text):
    issues = []
    if "bug" in text.lower():
        issues.append("bug")
    if "problem" in text.lower():
        issues.append("problem")
    if "issue" in text.lower():
        issues.append("issue")
    if "crash" in text.lower():
        issues.append("crash")
    return issues

def context_aware_sentiment_classification(text, keywords):
    positive_aspects = set()
    negative_aspects = set()
    
    for keyword in keywords:
        sentiment_result = analyze_sentiment(keyword)
        sentiment = sentiment_result['label']
        
        if sentiment == 'POSITIVE':
            positive_aspects.add(keyword)
        elif sentiment == 'NEGATIVE':
            negative_aspects.add(keyword)
    
    return positive_aspects, negative_aspects

def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        keywords = extract_keywords(review)
        pos_aspects, neg_aspects = context_aware_sentiment_classification(review, keywords)
        
        positive_aspects.update(pos_aspects)
        negative_aspects.update(neg_aspects)
        issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)

# Example reviews
reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]

# Extract insights
positive_aspects, negative_aspects, issues = extract_insights(reviews)

# Output results
print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

Positive Aspects: ['design is sleek and modern', 'Affordable', 'packaging', 'Highly recommended', 'offers', 'product', 'versatile', 'high', 'frequently', 'customer support team', 'eco-friendly', 'packaging was eco-friendly', 'product is a game-changer', 'easy to follow', 'intuitive', 'build quality', 'satisfied with the customer', 'satisfied with the customer service', 'advanced features', 'life', 'worth', 'love', 'product is durable and built', 'Impressed with the build quality', 'Numerous', 'product arrived', 'reasonable considering the features', 'extraordinary', 'Received', 'Responsive', 'making it user-friendly', 'user interface', 'product is durable', 'interface', 'features', 'Fantastic customer service', 'product exceeded', 'Great', 'Great value for the money', 'comprehensive and easy to follow', 'user experience', 'love this product', 'money', 'quality is top-notch', 'Excellent product', 'durable and built', 'exceeded my expectations', 'experience', 'shipping time', 'interface 

In [6]:
from transformers import pipeline
import yake

def analyze_sentiment(review: str):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5):
    # Using the initial keyword extraction approach
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=5, dedupLim=0.9, top=num_keywords, features=None)
    keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    return keywords

def extract_issues(text):
    issues = []
    if "bug" in text.lower():
        issues.append("bug")
    if "problem" in text.lower():
        issues.append("problem")
    if "issue" in text.lower():
        issues.append("issue")
    return issues

def context_aware_sentiment_classification(text, keywords):
    positive_aspects = set()
    negative_aspects = set()
    
    for keyword in keywords:
        sentiment_result = analyze_sentiment(keyword)
        sentiment = sentiment_result['label']
        
        if sentiment == 'POSITIVE':
            positive_aspects.add(keyword)
        elif sentiment == 'NEGATIVE':
            negative_aspects.add(keyword)
    
    return positive_aspects, negative_aspects

def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        keywords = extract_keywords(review)
        pos_aspects, neg_aspects = context_aware_sentiment_classification(review, keywords)
        
        positive_aspects.update(pos_aspects)
        negative_aspects.update(neg_aspects)
        issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)

# Example reviews
reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]

# Extract insights
positive_aspects, negative_aspects, issues = extract_insights(reviews)

# Output results
print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

c:\Users\Asif Sayyed\Documents\GitHub\MachineUnderstandsTextData\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Positive Aspects: ['design is sleek and modern', 'Affordable', 'packaging', 'Highly recommended', 'offers', 'product', 'versatile', 'high', 'frequently', 'customer support team', 'eco-friendly', 'packaging was eco-friendly', 'product is a game-changer', 'easy to follow', 'intuitive', 'build quality', 'satisfied with the customer', 'satisfied with the customer service', 'advanced features', 'life', 'worth', 'love', 'product is durable and built', 'Impressed with the build quality', 'Numerous', 'product arrived', 'reasonable considering the features', 'extraordinary', 'Received', 'Responsive', 'making it user-friendly', 'user interface', 'product is durable', 'interface', 'features', 'Fantastic customer service', 'product exceeded', 'Great', 'Great value for the money', 'comprehensive and easy to follow', 'user experience', 'love this product', 'money', 'quality is top-notch', 'Excellent product', 'durable and built', 'exceeded my expectations', 'experience', 'shipping time', 'interface 

In [12]:
from transformers import pipeline
import yake

def analyze_sentiment(review: str):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(review)
    return result[0]

def extract_keywords(text, num_keywords=5):
    # Using the initial keyword extraction approach
    custom_kw_extractor = yake.KeywordExtractor(lan="en", n=5, dedupLim=0.9, top=num_keywords, features=None)
    keywords = [kw for kw, score in custom_kw_extractor.extract_keywords(text)]
    return keywords

def extract_issues(text):
    issues = []
    if "bug" in text.lower():
        issues.append("bug")
    if "problem" in text.lower():
        issues.append("problem")
    if "issue" in text.lower():
        issues.append("issue")
    if "crash" in text.lower():
        issues.append("crash")
    return issues

def context_aware_sentiment_classification(text, keywords):
    positive_aspects = set()
    negative_aspects = set()
    
    for keyword in keywords:
        sentiment_result = analyze_sentiment(keyword)
        sentiment = sentiment_result['label']
        
        if sentiment == 'POSITIVE':
            positive_aspects.add(keyword)
        elif sentiment == 'NEGATIVE':
            negative_aspects.add(keyword)
    
    return positive_aspects, negative_aspects

@log_runtime_to_csv(n=10, filename='updated_approach_runtimes.csv')
def extract_insights(reviews):
    positive_aspects = set()
    negative_aspects = set()
    issues = set()

    for review in reviews:
        keywords = extract_keywords(review)
        pos_aspects, neg_aspects = context_aware_sentiment_classification(review, keywords)
        
        positive_aspects.update(pos_aspects)
        negative_aspects.update(neg_aspects)
        issues.update(extract_issues(review))

    return list(positive_aspects), list(negative_aspects), list(issues)

# Example reviews
reviews = [
    "I love this product! It's amazing!",
    "The quality is top-notch, but the price is too high.",
    "The instructions are unclear and confusing.",
    "Great features, but it crashes frequently.",
    "Not satisfied with the customer service.",
    "Excellent product, worth every penny!",
    "The design is sleek and modern, but the battery life is disappointing.",
    "Easy to use and setup, but lacks some advanced features.",
    "Disappointed with the shipping time. It took longer than expected.",
    "This product exceeded my expectations. Highly recommended!",
    "The customer support team was helpful and resolved my issue promptly.",
    "The user interface is intuitive, making it user-friendly.",
    "Received a damaged product. Poor packaging.",
    "The price is reasonable considering the features it offers.",
    "I encountered a software bug that needs immediate attention.",
    "The sound quality is excellent, but the device heats up quickly.",
    "The product is durable and built to last. Impressed with the build quality.",
    "Terrible experience with the return process. It was a hassle.",
    "Average product. Nothing extraordinary, but it gets the job done.",
    "The user manual is comprehensive and easy to follow.",
    "Not recommended. Numerous issues with functionality.",
    "The product arrived earlier than expected. Pleasant surprise!",
    "The interface is outdated and could use a modern redesign.",
    "Fantastic customer service. They went above and beyond to assist me.",
    "The product is versatile and can be used for various applications.",
    "Not happy with the performance. Laggy and slow.",
    "Great value for the money. Affordable and high-quality.",
    "The product is a game-changer. I can't imagine life without it.",
    "The packaging was eco-friendly, which is a plus for me.",
    "Too many unnecessary features. It complicates the user experience.",
    "Responsive touchscreen, but the battery drains quickly.",
]

# Extract insights
positive_aspects, negative_aspects, issues = extract_insights(reviews)

# Output results    
print("Positive Aspects:", positive_aspects)
print("Negative Aspects:", negative_aspects)
print("Issues:", issues)

Average runtime over 10 runs: 107.8121 seconds
Positive Aspects: ['design is sleek and modern', 'Affordable', 'packaging', 'Highly recommended', 'offers', 'product', 'versatile', 'high', 'frequently', 'customer support team', 'eco-friendly', 'packaging was eco-friendly', 'product is a game-changer', 'easy to follow', 'intuitive', 'build quality', 'satisfied with the customer', 'satisfied with the customer service', 'advanced features', 'life', 'worth', 'love', 'product is durable and built', 'Impressed with the build quality', 'Numerous', 'product arrived', 'reasonable considering the features', 'extraordinary', 'Received', 'Responsive', 'making it user-friendly', 'user interface', 'product is durable', 'interface', 'features', 'Fantastic customer service', 'product exceeded', 'Great', 'Great value for the money', 'comprehensive and easy to follow', 'user experience', 'love this product', 'money', 'quality is top-notch', 'Excellent product', 'durable and built', 'exceeded my expectatio